# 循环神经网络 --- 使用Gluon

本节介绍如何使用`Gluon`训练循环神经网络。


## Penn Tree Bank (PTB) 数据集

我们以单词为基本元素来训练语言模型。[Penn Tree Bank](https://catalog.ldc.upenn.edu/ldc99t42)（PTB）是一个标准的文本序列数据集。它包括训练集、验证集和测试集。

下面我们载入数据集。

In [3]:
import math 
import os 
import time 
import numpy as np 
import mxnet as mx 
from mxnet import gluon,autograd 
from mxnet.gluon import nn,rnn 
import zipfile
with zipfile.ZipFile('../data/ptb.zip','r') as zin:
    zin.extractall('../data/') 


## 建立词语索引

下面定义了`Dictionary`类来映射词语和索引。


In [8]:
class Dictionary(object):
    def __init__(self):
        self.word_to_idx={}
        self.idx_to_word=[]
    
    def add_word(self,word):
        if word not in self.word_to_idx:
            self.idx_to_word.append(word) 
            self.word_to_idx[word]=len(self.idx_to_word)-1
        return self.word_to_idx[word] 
    def __len__(self):
        return len(self.idx_to_word)

以下的`Corpus`类按照读取的文本数据集建立映射词语和索引的词典，并将文本转换成词语索引的序列。这样，每个文本数据集就变成了`NDArray`格式的整数序列。


In [9]:
class Corpus(object):
    def __init__(self,path):
        self.dictionary=Dictionary() 
        self.train=self.tokenize(path+'train.txt') 
        self.valid=self.tokenize(path+'valid.txt') 
        self.test=self.tokenize(path+'test.txt') 
    def tokenize(self,path):
        assert os.path.exists(path) 
        # 将词语添加至词典。 
        with open(path,'r') as f: 
            tokens=0 
            for line in f:
                words=line.split()+['<eos>']
                tokens+=len(words)
                for word in words:
                    self.dictionary.add_word(word)
        # 将文本转换成词语索引的序列（NDArray 格式） 
        with open(path,'r') as f:
            indices=np.zeros((tokens,),dtype='int32') 
            idx=0 
            for line in f:
                words=line.split()+['<eos>']
                for word in words:
                    indices[idx]=self.dictionary.word_to_idx[word] 
                    idx+=1 
        return mx.nd.array(indices,dtype='int32')

In [10]:
data='../data/ptb/ptb.'
corpus=Corpus(data) 
vocab_size=len(corpus.dictionary)
vocab_size

10000

我们可以定义一个循环神经网络模型库。这样就可以支持各种不同的循环神经网络模型了 。 

In [12]:
class RNNModel(gluon.Block):
    """ 循环神经网络模型库"""
    def __init__(self,mode,vocab_size,embed_dim,hidden_dim,
                 num_layers,dropout=0.5,**kwargs):
        super(RNNModel,self).__init__(**kwargs)
        with self.name_scope():
            self.drop=nn.Dropout(dropout) 
            self.encoder=nn.Embedding(vocab_size,embed_dim,
                                     weight_initializer=mx.init.Uniform(0.1))
            if mode=='rnn_relu':
                self.rnn=rnn.RNN(hiden_dim,num_layers,activation='relu',
                                dropout=dropout,input_size=embed_dim) 
            elif mode=='rnn_tanh':
                self.rnn=rnn.RNN(hidden_dim,num_layers,dropout=droptout,
                                input_size=embed_dim) 
            elif mode=='lstm':
                self.nn=rnn.LSTM(hidden_dim,num_layers,dropout=dropout,
                                input_size=embed_dim) 
            elif mode=='gru':
                self.rnn=rnn.GRU(hidden_dim,num_layers,dropout=dropout,
                                input_size=embed_dim) 
            else:
                raise ValueError('Invalid mode %s. Options are rnn_relu,rnn_tanh,lstm and gru'%mode) 
            self.decoder=nn.Dense(vocab_size,in_units=hidden_dim) 
            self.hidden_dim=hidden_dim 
    def forward(self,inputs,state):
        emb=self.drop(self.encoder(inputs)) 
        output,state=self.rnn(emb,state) 
        output=self.drop(output) 
        decoded=self.decoder(output.reshape((-1,self.hidden_dim)))
        return decoded,state 
    def begin_state(self,*args,**kwargs):
        return self.rnn.begin_state(*args,**kwargs) 



## 定义参数

我们接着定义模型参数。我们选择使用ReLU为激活函数的循环神经网络为例。这里我们把`epochs`设为1是为了演示方便。


## 多层循环神经网络

我们通过`num_layers`设置循环神经网络隐含层的层数，例如2。

对于一个多层循环神经网络，当前时刻隐含层的输入来自同一时刻输入层（如果有）或上一隐含层的输出。每一层的隐含状态只沿着同一层传递。

把[单层循环神经网络](rnn-scratch.md)中隐含层的每个单元当做一个函数$f$，这个函数在$t$时刻的输入是$\mathbf{X}_t, \mathbf{H}_{t-1}$，输出是$\mathbf{H}_t$：

$$f(\mathbf{X}_t, \mathbf{H}_{t-1}) = \mathbf{H}_t$$

假设输入为第0层，输出为第$L+1$层，在一共$L$个隐含层的循环神经网络中，上式中可以拓展成以下的函数:

$$f(\mathbf{H}_t^{(l-1)}, \mathbf{H}_{t-1}^{(l)}) = \mathbf{H}_t^{(l)}$$

如下图所示。

![](../img/multi-layer-rnn.svg)
